In [ ]:
import pandas as pd
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.cluster import OPTICS, cluster_optics_dbscan

# OPTICS

In [ ]:
X = pd.read_pickle('./data/unlabeled_behavior.pkl')